In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler,StandardScaler

import matplotlib.pyplot as plt
import seaborn as sbn

train_data=pd.read_csv('train_data.csv')
train_data['type']='train'
test=pd.read_csv('test_a.csv')
test['tradeMoney']=0
test['type']='test'

real_data=pd.read_csv('sub_a_913.csv')

In [2]:
#删除异常数据

#删除结合区域面积和交易金额的异常数据
#根据上述图，我们可以将一定范围内的异常点进行清除
def deep_delete(data):
    print("删除前：",data.shape[0])
    #删除tradeMoney等于0的数据
    data.drop(data[(data['tradeMoney']==0)&(data['type']=='train')].index,inplace=True)
    print("交易金额等于0的删除后{}".format(data.shape[0]))
    data.drop(data[(data['region']=='RG00001')&((data['tradeMoney']>50000)|(data['area']>200))].index,inplace=True)
    data.drop(data[(data['region']=='RG00002')&((data['tradeMoney']>100000)|(data['area']>500))].index,inplace=True)
    data.drop(data[(data['region']=='RG00003')&((data['tradeMoney']>50000)|(data['area']>400))].index,inplace=True)
    data.drop(data[(data['region']=='RG00007')&((data['tradeMoney']>50000)|(data['area']>200))].index,inplace=True)
    data.drop(data[(data['region']=='RG00010')&((data['tradeMoney']>50000)|(data['area']>500))].index,inplace=True)
    data.drop(data[(data['region']=='RG00008')&((data['tradeMoney']>50000)|(data['area']>200))].index,inplace=True)
    data.drop(data[(data['region']=='RG00004')&((data['tradeMoney']>30000)|(data['area']>250))].index,inplace=True)
    data.drop(data[(data['region']=='RG00012')&((data['tradeMoney']>50000)|(data['area']>200))].index,inplace=True)
    data.drop(data[(data['region']=='RG00009')&((data['tradeMoney']>25000)|(data['area']>200))].index,inplace=True)
    data.drop(data[(data['region']=='RG00011')&((data['tradeMoney']>40000)|(data['area']>500))].index,inplace=True)
    data.drop(data[(data['region']=='RG00013')&((data['tradeMoney']>40000)|(data['area']>250))].index,inplace=True)
    data.drop(data[(data['region']=='RG00014')&((data['tradeMoney']>20000)|(data['area']>250))].index,inplace=True)
    data.drop(data[(data['region']=='RG00005')&((data['tradeMoney']>50000)|(data['area']>400))].index,inplace=True)
    return data

train_data=deep_delete(train_data)
print("区域面积金额删除后{}".format(train_data.shape[0]))

删除前： 41440
交易金额等于0的删除后41431
区域面积金额删除后41327


In [3]:
#提取每平方均价特征
#测试集按小区的平均值进行构建
'''
train_data['ave_money']=train_data['tradeMoney']/train_data['area']
train_d=train_data.groupby(['communityName'])[['ave_money']].mean().reset_index()
test=test.merge(train_d,on=['communityName'],how='left')
'''

"\ntrain_data['ave_money']=train_data['tradeMoney']/train_data['area']\ntrain_d=train_data.groupby(['communityName'])[['ave_money']].mean().reset_index()\ntest=test.merge(train_d,on=['communityName'],how='left')\n"

In [4]:
data=pd.concat([train_data,test],axis=0)
data

,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,city,...,landMeanPrice,totalWorkers,newWorkers,residentPopulation,pv,uv,lookNum,tradeTime,tradeMoney,type
0,100309852,68.06,未知方式,2室1厅1卫,低,16,暂无数据,其他,XQ00051,SH,...,0.000000,28248,614,111546,1124.0,284.0,0,2018/11/28,2000.0,train
1,100307942,125.55,未知方式,3室2厅2卫,中,14,暂无数据,简装,XQ00130,SH,...,0.000000,14823,148,157552,701.0,22.0,1,2018/12/16,2000.0,train
2,100307764,132.00,未知方式,3室2厅2卫,低,32,暂无数据,其他,XQ00179,SH,...,0.000000,77645,520,131744,57.0,20.0,1,2018/12/22,16000.0,train
3,100306518,57.00,未知方式,1室1厅1卫,中,17,暂无数据,精装,XQ00313,SH,...,3080.033100,8750,1665,253337,888.0,279.0,9,2018/12/21,1600.0,train
4,100305262,129.00,未知方式,3室2厅3卫,低,2,暂无数据,毛坯,XQ01257,SH,...,0.000000,800,117,125309,2038.0,480.0,0,2018/11/18,2900.0,train
5,100304268,223.35,未知方式,4室3厅3卫,低,2,暂无数据,其他,XQ00457,SH,...,0.000000,48204,44561,719428,4832.0,1304.0,0,2018/11/14,14000.0,train
6,100304055,78.94,未知方式,2室2厅1卫,中,18,暂无数据,其他,XQ00246,SH,...,0.000000,8750,1467,253337,1586.0,469.0,0,2018/11/18,1000.0,train
7,100295123,76.00,未知方式,2室1厅1卫,中,5,暂无数据,其他,XQ02331,SH,...,0.000000,28546,0,371900,40787.0,2942.0,0,2018/3/24,1800.0,train
8,100293141,120.55,未知方式,3室2厅1卫,中,6,暂无数据,其他,XQ02346,SH,...,0.000000,28546,0,371900,40787.0,2942.0,0,2018/3/19,1450.0,train
9,100293063,83.00,未知方式,3室1厅1卫,中,5,暂无数据,其他,XQ02310,SH,...,0.000000,28546,0,371900,34441.0,3035.0,0,2018/4/26,1700.0,train


In [5]:
# tradeTime 可以分为年月日,因为都是18年的数据，所以提取月日数据特征就好
# pv ，uv 先用均值填充
# 将"rentType"缺失值用众数进行填充
data['rentType'][data['rentType']=='--']='未知方式'

def months(d):
    s=d.split('/')
    return int(s[1])

def days(d):
    s=d.split('/')
    return int(s[2])
data['month']=data['tradeTime'].apply(months)
data['days']=data['tradeTime'].apply(days)

data['pv']=data['pv'].fillna(data[data['pv'].notnull()]['pv'].mean())
data['uv']=data['uv'].fillna(data[data['uv'].notnull()]['uv'].mean())
data[data['pv'].notnull()]

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,city,...,newWorkers,residentPopulation,pv,uv,lookNum,tradeTime,tradeMoney,type,month,days
0,100309852,68.06,未知方式,2室1厅1卫,低,16,暂无数据,其他,XQ00051,SH,...,614,111546,1124.0,284.0,0,2018/11/28,2000.0,train,11,28
1,100307942,125.55,未知方式,3室2厅2卫,中,14,暂无数据,简装,XQ00130,SH,...,148,157552,701.0,22.0,1,2018/12/16,2000.0,train,12,16
2,100307764,132.00,未知方式,3室2厅2卫,低,32,暂无数据,其他,XQ00179,SH,...,520,131744,57.0,20.0,1,2018/12/22,16000.0,train,12,22
3,100306518,57.00,未知方式,1室1厅1卫,中,17,暂无数据,精装,XQ00313,SH,...,1665,253337,888.0,279.0,9,2018/12/21,1600.0,train,12,21
4,100305262,129.00,未知方式,3室2厅3卫,低,2,暂无数据,毛坯,XQ01257,SH,...,117,125309,2038.0,480.0,0,2018/11/18,2900.0,train,11,18
5,100304268,223.35,未知方式,4室3厅3卫,低,2,暂无数据,其他,XQ00457,SH,...,44561,719428,4832.0,1304.0,0,2018/11/14,14000.0,train,11,14
6,100304055,78.94,未知方式,2室2厅1卫,中,18,暂无数据,其他,XQ00246,SH,...,1467,253337,1586.0,469.0,0,2018/11/18,1000.0,train,11,18
7,100295123,76.00,未知方式,2室1厅1卫,中,5,暂无数据,其他,XQ02331,SH,...,0,371900,40787.0,2942.0,0,2018/3/24,1800.0,train,3,24
8,100293141,120.55,未知方式,3室2厅1卫,中,6,暂无数据,其他,XQ02346,SH,...,0,371900,40787.0,2942.0,0,2018/3/19,1450.0,train,3,19
9,100293063,83.00,未知方式,3室1厅1卫,中,5,暂无数据,其他,XQ02310,SH,...,0,371900,34441.0,3035.0,0,2018/4/26,1700.0,train,4,26


In [6]:
#提取房间数量特征
def room_num(room):
    return int(room[0])

data['room_num']=data['houseType'].apply(room_num)

#提取大厅数量特征
def lobby_num(lobby):
    return int(lobby[2])

data['lobby_num']=data['houseType'].apply(lobby_num)

#提取卫生间数量
def toilet_num(toilet):
    return int(toilet[4])

data['toilet_num']=data['houseType'].apply(toilet_num)

#填充租房类型
data.loc[(data['rentType']=='未知方式')&(data['room_num']<=1),'rentType']='整租'

data

,ID,area,rentType,houseType,houseFloor,totalFloor,houseToward,houseDecoration,communityName,city,...,uv,lookNum,tradeTime,tradeMoney,type,month,days,room_num,lobby_num,toilet_num
0,100309852,68.06,未知方式,2室1厅1卫,低,16,暂无数据,其他,XQ00051,SH,...,284.0,0,2018/11/28,2000.0,train,11,28,2,1,1
1,100307942,125.55,未知方式,3室2厅2卫,中,14,暂无数据,简装,XQ00130,SH,...,22.0,1,2018/12/16,2000.0,train,12,16,3,2,2
2,100307764,132.00,未知方式,3室2厅2卫,低,32,暂无数据,其他,XQ00179,SH,...,20.0,1,2018/12/22,16000.0,train,12,22,3,2,2
3,100306518,57.00,整租,1室1厅1卫,中,17,暂无数据,精装,XQ00313,SH,...,279.0,9,2018/12/21,1600.0,train,12,21,1,1,1
4,100305262,129.00,未知方式,3室2厅3卫,低,2,暂无数据,毛坯,XQ01257,SH,...,480.0,0,2018/11/18,2900.0,train,11,18,3,2,3
5,100304268,223.35,未知方式,4室3厅3卫,低,2,暂无数据,其他,XQ00457,SH,...,1304.0,0,2018/11/14,14000.0,train,11,14,4,3,3
6,100304055,78.94,未知方式,2室2厅1卫,中,18,暂无数据,其他,XQ00246,SH,...,469.0,0,2018/11/18,1000.0,train,11,18,2,2,1
7,100295123,76.00,未知方式,2室1厅1卫,中,5,暂无数据,其他,XQ02331,SH,...,2942.0,0,2018/3/24,1800.0,train,3,24,2,1,1
8,100293141,120.55,未知方式,3室2厅1卫,中,6,暂无数据,其他,XQ02346,SH,...,2942.0,0,2018/3/19,1450.0,train,3,19,3,2,1
9,100293063,83.00,未知方式,3室1厅1卫,中,5,暂无数据,其他,XQ02310,SH,...,3035.0,0,2018/4/26,1700.0,train,4,26,3,1,1


In [7]:
#对房间的类型进行特征交叉
#这里看出一室一厅更受欢迎

data['room_lobby']=data['room_num'].astype('str')+'_'+data['lobby_num'].astype('str')
data['room_toilet']=data['room_num'].astype('str')+'_'+data['toilet_num'].astype('str')
data['lobby_toilet']=data['lobby_num'].astype('str')+'_'+data['toilet_num'].astype('str')
print(data['room_lobby'].value_counts())
print(data['room_toilet'].value_counts())
print(data['room_toilet'].value_counts())

1_1    10430
2_1     9321
2_2     8222
3_2     5155
3_1     4241
4_1     2002
1_0     1453
1_2      998
4_2      810
2_0      473
5_1      206
5_2      126
4_3       85
6_1       66
5_3       63
6_2       29
3_3       29
3_0       24
7_1       18
6_3       10
4_0        6
7_2        6
5_4        6
2_3        4
6_4        3
4_4        1
8_2        1
6_0        1
8_3        1
9_2        1
5_5        1
3_4        1
8_4        1
7_3        1
7_4        1
Name: room_lobby, dtype: int64
2_1    16874
1_1    12718
3_1     6158
3_2     3106
4_1     2013
2_2     1116
4_2      487
4_3      301
5_1      213
3_3      167
1_0      103
4_4       94
5_3       80
6_1       68
1_2       60
5_4       47
5_2       45
2_0       22
6_4       18
7_1       18
5_5       16
3_4       15
4_5        9
2_3        8
6_3        8
6_2        7
6_5        4
7_3        4
7_4        3
3_0        3
6_7        3
8_4        2
6_6        1
5_6        1
3_5        1
8_3        1
9_5        1
7_7        1
Name: room_toilet, d

## 特征合并

In [ ]:
def merge(data):
    data['transport']=data['subwayStationNum']/data['subwayStationNum'].mean()+2*data['busStationNum']/data['busStationNum'].mean()
    data['school']=data['interSchoolNum']/data['interSchoolNum'].mean()+data['schoolNum']/data['schoolNum'].mean()
                    +data['privateSchoolNum']/data['privateSchoolNum'].mean()
    

In [8]:
#根据分析，可以将数据属性分为类别特征和连续型特征
#类别特征
category_feauture=[ 'rentType', 'houseType', 'houseFloor','houseToward', 'houseDecoration', 'communityName', 'region',
                    'plate', 'buildYear', 'room_lobby', 'room_toilet', 'lobby_toilet']

#连续型特征
Value_feauture=[f for f in data.columns if f not in category_feauture and f not in ['ID','tradeTime','tradeMoney','type','city']]
print(Value_feauture)

target=['tradeMoney']

['area', 'totalFloor', 'saleSecHouseNum', 'subwayStationNum', 'busStationNum', 'interSchoolNum', 'schoolNum', 'privateSchoolNum', 'hospitalNum', 'drugStoreNum', 'gymNum', 'bankNum', 'shopNum', 'parkNum', 'mallNum', 'superMarketNum', 'totalTradeMoney', 'totalTradeArea', 'tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney', 'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum', 'supplyNewNum', 'supplyLandNum', 'supplyLandArea', 'tradeLandNum', 'tradeLandArea', 'landTotalPrice', 'landMeanPrice', 'totalWorkers', 'newWorkers', 'residentPopulation', 'pv', 'uv', 'lookNum', 'month', 'days', 'room_num', 'lobby_num', 'toilet_num']


In [9]:
for f in category_feauture:
    ln=LabelEncoder()
    data[f]=ln.fit_transform(data[f])
    
for f in Value_feauture:
    if f not in ['ave_money']:
        mms=MinMaxScaler(feature_range=(0,1))
        data[f]=mms.fit_transform(data[f].values.reshape(-1,1))

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [10]:
feas=category_feauture+Value_feauture

In [11]:
#划分数据集

train_=data[data['type']=='train'][feas]
test_=data[data['type']=='test'][feas]
train_target=data[data['type']=='train']['tradeMoney']
test_target=data[data['type']=='test']['tradeMoney']

#划分数据为训练集和验证集，比例为 8:2

train_x,valid_x,train_y,valid_y=train_test_split(train_,train_target,test_size=0.2,random_state=42)


In [12]:
#构建baseline模型
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 63,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_seed':0,
    'bagging_freq': 1,
    'verbose': 1,
    'reg_alpha':1,
    'reg_lambda':2,
    'min_child_weight':6
}

lgb_train=lgb.Dataset(train_x,train_y)
lgb_eval=lgb.Dataset(valid_x,valid_y,reference=lgb_train)

gbm=lgb.train(params,lgb_train,num_boost_round=10000,valid_sets=[lgb_train,lgb_eval],
                valid_names=['train','valid'],
                early_stopping_rounds=2000,
                verbose_eval=1000,
                )
valid=gbm.predict(valid_x)
feature_rank=gbm.feature_importance()
features_df=pd.DataFrame({ 'column':feas,'importance': feature_rank}).sort_values(by='importance', ascending=False)
print(features_df)

Train until valid scores didn't improve in 2000 rounds.
[1000]	train's l1: 549.684	valid's l1: 688.377
[2000]	train's l1: 471.03	valid's l1: 667.627
[3000]	train's l1: 421.119	valid's l1: 659.067
[4000]	train's l1: 383.684	valid's l1: 655.135
[5000]	train's l1: 353.7	valid's l1: 653.124
[6000]	train's l1: 328.274	valid's l1: 651.843
[7000]	train's l1: 306.268	valid's l1: 651.485
[8000]	train's l1: 286.735	valid's l1: 651.397
[9000]	train's l1: 269.529	valid's l1: 651.661
Early stopping, best iteration is:
[7767]	train's l1: 291.005	valid's l1: 651.25
                column  importance
12                area       61244
51                days       42680
8            buildYear       34422
13          totalFloor       32098
5        communityName       26595
34   tradeNewMeanPrice       17923
30      tradeMeanPrice       16225
32  totalNewTradeMoney       13017
1            houseType       12698
36        remainNewNum       11729
28     totalTradeMoney       11655
14     saleSecHouseNum 

In [13]:
val=valid_x[['area']]
val['pred']=valid
val['pre']=valid_y
def score(real_data):
    a=sum((real_data['pred']-real_data['pre'])*(real_data['pred']-real_data['pre']))
    b=sum((real_data['pre']-real_data['pre'].mean())*(real_data['pre']-real_data['pre'].mean()))
    s=1-a/b
    return s

val_s=score(val)
val_s
# 0.8654572967842439

#做了归一化之后 0.8661834939412152

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0.8673970483488771

In [14]:
#训练整个数据模型，并预测

lgb_train_=lgb.Dataset(train_,train_target)
gbm=lgb.train(params,lgb_train_,num_boost_round=10000,valid_sets=[lgb_train_],
              valid_names=['train'],
              early_stopping_rounds=2000,
              verbose_eval=1000)

real_data['pred']=gbm.predict(test_)

Train until valid scores didn't improve in 2000 rounds.
[1000]	train's l1: 563.88
[2000]	train's l1: 486.565
[3000]	train's l1: 438.275
[4000]	train's l1: 403.468
[5000]	train's l1: 375.484
[6000]	train's l1: 351.573
[7000]	train's l1: 331.09
[8000]	train's l1: 312.881
[9000]	train's l1: 296.631
[10000]	train's l1: 281.623


#  测试结果记录

#未做很多的特征工程，只是补缺失值，然后将类别特征进行了处理得到结果：  0.40585301363216797

#删除异常数据后，进行预测结果：0.9211771459530279 ，这只是清除了异常数据就这么高???????是不是有点假??????

#做了归一化之后 结果是：0.9214346671924497

#加了小区的平均房价 ：-1.5635201470692257？？？？？？

In [15]:
def score(real_data):
    a=sum((real_data['pred']-real_data['pre'])*(real_data['pred']-real_data['pre']))
    b=sum((real_data['pre']-real_data['pre'].mean())*(real_data['pre']-real_data['pre'].mean()))
    s=1-a/b
    return s

s=score(real_data)
s

0.9220962828170749